In [23]:
import mlflow
from mlflow.tracking import MlflowClient

In [24]:
mlflow_arn = "arn:aws:sagemaker:eu-west-1:575618486322:mlflow-tracking-server/dev-mlflow"
mlflow_experiment_name = "02-sample-experiment"
target_run_name = "my-custom-name"  

In [25]:
mlflow.set_tracking_uri(mlflow_arn)
mlflow.set_experiment(mlflow_experiment_name)

<Experiment: artifact_location='s3://ipf-sds-datalake-dev-data-science-bucket/mlflow/2', creation_time=1730285259326, experiment_id='2', last_update_time=1730285259326, lifecycle_stage='active', name='02-sample-experiment', tags={}>

# GET RUNS

In [12]:
# Initialize MLflow client
client = MlflowClient()

# Get the experiment ID from the experiment name
experiment = client.get_experiment_by_name(mlflow_experiment_name)
if experiment is None:
    raise ValueError(f"Experiment '{mlflow_experiment_name}' not found.")
experiment_id = experiment.experiment_id

# List all runs in the experiment
runs = client.search_runs(experiment_ids=[experiment_id])

# DELETE RUNS

In [15]:
# Filter and delete runs with the target name
for run in runs:
    if run.data.tags.get("mlflow.runName") == target_run_name:
        run_id = run.info.run_id
        client.delete_run(run_id)
        print(f"Deleted run with ID: {run_id} and name: {target_run_name}")

# RESTORE A RUN

In [16]:
client.restore_run(run_id)
print(f"Run with ID {run_id} has been restored.")

Run with ID 068851125fb2432e93cac46acb81daae has been restored.


# LIST DELETED RUNS

In [17]:
from mlflow.entities import ViewType

experiment = client.get_experiment_by_name(mlflow_experiment_name)
experiment_id = experiment.experiment_id

# Search for deleted runs in the experiment
deleted_runs = client.search_runs(
    experiment_ids=[experiment_id], 
    run_view_type=ViewType.DELETED_ONLY
)

# Print details of each deleted run
print("Deleted runs:")
for run in deleted_runs:
    print(f"Run ID: {run.info.run_id}")
    print(f"Run Name: {run.data.tags.get('mlflow.runName', 'N/A')}")
    print(f"Deletion Time: {run.info.end_time}")  # end_time is when the run was deleted
    print("-" * 40)

Deleted runs:
Run ID: 97eac6e056ee4f5c9fc5453bb4bbe134
Run Name: my-custom-name
Deletion Time: 1730288566977
----------------------------------------
Run ID: 6484bcf3f33e431c87b6774454d89567
Run Name: my-custom-name
Deletion Time: 1730288307261
----------------------------------------
